In [ ]:
import os
from mistralai import Mistral

api_key = "zdoj0CXuYD1JXWJTKvNAicpC0YOnLaak"
client = Mistral(api_key=api_key)


In [43]:
def parse_pdf_offline(pdf_path):
    uploaded_pdf = upload_pdf(pdf_path)
    signed_url = client.files.get_signed_url(file_id=uploaded_pdf.id)
    ocr_response = client.ocr.process(
    model="mistral-ocr-latest",
        document={
            "type": "document_url",
            "document_url": signed_url.url,
        },
        include_image_base64=True
    )
    return ocr_response
def upload_pdf(pdf_path):
    uploaded_pdf = client.files.upload(
        file={
            "file_name": "uploaded_file.pdf",
            "content": open(pdf_path, "rb"),
        },
        purpose="ocr"
    )  
    return uploaded_pdf


In [44]:
import base64
def save_base64_image(base64_data, output_path='output_image.jpg'):
    try:
        if base64_data.startswith('data:image'):
            base64_data = base64_data.split(',', 1)[1]
        image_bytes = base64.b64decode(base64_data)
        with open(output_path, 'wb') as f:
            f.write(image_bytes)
        print(f"图片已保存到 {os.path.abspath(output_path)}")
    except Exception as e:
        print(f"保存图片时出错: {e}")

In [36]:
ocr_response = parse_pdf_offline("./2201.04234v3.pdf")

In [ ]:
import os
for page in ocr_response.pages:    
    print(page.dict().keys())
    print(page.index)
    images = page.dict()["images"]
    if images:
        for image in images:
            save_img_name = str(page.index) + "-" + image["id"]
            save_base64_image(image["image_base64"], os.path.join(os.getcwd(),save_img_name ))